In [2]:
import numpy as np
import random

In [3]:
!nvidia-smi

Tue May 11 02:02:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
%pwd

'/content'

In [6]:
# output = np.array([1, 2, 3])
# with open('save_output.npy', 'wb') as f:
#   np.save(f, output)

In [7]:
%ls

gdrive/  sample_data/


In [8]:
#!pip install kaggle
# !mkdir .kaggle

In [9]:
# import json
# token = {"username":"sreedhar1895","key":"c92f28215ef8f502a45301d532016efc"}
# with open('/content/.kaggle/kaggle.json', 'w') as file:
#   json.dump(token, file)

In [10]:
# !chmod 600 /content/.kaggle/kaggle.json
# !cp /content/.kaggle/kaggle.json /root/.kaggle/
# !kaggle config set -n path -v /content

In [11]:
%pwd

'/content'

In [12]:
# !kaggle competitions download -c 11785-spring2021-hw1p2

# Connection to Kaggle Completed and Data Loaded


In [13]:
%pwd
%ls

gdrive/  sample_data/


In [14]:
# %cd competitions/11785-spring2021-hw1p2/

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
%ls

ls: cannot access 'gdrive': Transport endpoint is not connected
drive/  gdrive/  sample_data/


In [17]:
import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from sklearn.metrics import accuracy_score

In [18]:
# Check if cuda is available
cuda = torch.cuda.is_available()
num_workers = 8 if cuda else 0
print("Cuda = " + str(cuda) + "with num_workers = " + str(num_workers))

Cuda = Truewith num_workers = 8


In [19]:
drivepath = '/content/drive/MyDrive/DL_Group_Project'
datapath = '/content/drive/MyDrive/DL_Group_Project/Dataset/Preprocessed_Data'
drivepath_final = '/content/drive/MyDrive/DL_Group_Project/experiments/baseline'
BATCH_SIZE = 64

In [20]:
class PhonemesDataset(Dataset):

  # X - Frame, Features (For all phonemes)
  # Y - Frame, Labels (For all phonemes)
  # Softmax in the end
    
    def __init__(self, basepath, mode):
      phoneme_features = np.zeros((1, 40))  # eliminate this row
      phoneme_labels = np.zeros((1))  # eliminate this row

      with os.scandir(basepath) as entries:
        for entry in entries:
          if entry.is_file():
            if "features" in entry.name and mode in entry.name:
              phoneme_tag = entry.name.split("_")[0]

              features_filepath = entry.path
              labels_filepath = f"{basepath}/{phoneme_tag}_{mode}_labels.npy"

              other_phoneme_features = np.load(features_filepath, allow_pickle=True)
              other_phoneme_labels = np.load(labels_filepath, allow_pickle=True)
              
              # stack to phoneme features
              phoneme_features = np.concatenate((phoneme_features, other_phoneme_features))
              phoneme_labels = np.concatenate((phoneme_labels, other_phoneme_labels))
              
      self.X = phoneme_features[1:]
      self.Y = phoneme_labels[1:]

    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, index):
        x = torch.Tensor(self.X[index]).float()
        y = torch.as_tensor(self.Y[index]).long()
    
        return x,y

    def get_phoneme_label_frames(self, phoneme_label):
        phoneme_indexes = np.where(self.Y == phoneme_label)
        return torch.Tensor(self.X[phoneme_indexes]).float(), torch.as_tensor(self.Y[phoneme_indexes]).long()

Reading the Data

In [21]:
def make_dataloader(dataset, train, batch_size):
  if train:
    shuffle = True
    drop_last = True
  else:
    shuffle = False
    drop_last = False
    
  loader = DataLoader(dataset=dataset, batch_size=batch_size,
                      drop_last=drop_last, shuffle=shuffle,
                      pin_memory=True, num_workers=8)
  
  return loader

In [22]:
train_data = PhonemesDataset(basepath=datapath, mode="train")
train_loader = make_dataloader(dataset=train_data, train=True, batch_size=BATCH_SIZE)
print(f"train_data.shape: {train_data.X.shape}")

dev_data = PhonemesDataset(basepath=datapath, mode="dev")
dev_loader = make_dataloader(dataset=dev_data, train=True, batch_size=BATCH_SIZE)
print(f"dev_data.shape: {dev_data.X.shape}")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train_data.shape: (53755, 40)
dev_data.shape: (11535, 40)


In [23]:
for batch_number, (data_x, data_y) in enumerate(dev_loader):
  print("Batch Number ", batch_number)
  print("Data Features", data_x)
  print("Data Label", data_y)

print("test")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Batch Number  0
Data Features tensor([[-14.4972, -13.7209, -14.2334,  ..., -14.0044, -15.0242, -14.2785],
        [-15.4104, -16.5470, -17.0770,  ..., -14.1436, -13.4784, -13.9817],
        [ -9.6064,  -9.4234, -10.1894,  ..., -12.3197, -11.9232, -11.6337],
        ...,
        [-12.4734, -12.0244, -12.3411,  ..., -13.7613, -12.7276, -13.3693],
        [-17.3070, -17.6775, -17.5034,  ..., -12.8646, -13.4547, -13.2644],
        [-15.8515, -16.5097, -16.6809,  ..., -15.1411, -14.1546, -13.6293]])
Data Label tensor([ 0,  1, 22,  0, 13,  0,  0,  0,  0,  0,  0,  0,  1, 19, 14, 27,  0, 19,
        23,  0, 18,  0,  0,  0,  0,  0, 21, 17, 30, 15, 18, 33,  0, 33,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 10, 13, 22, 32,  0,  0, 11,  0, 34,  0,
         0,  0,  0, 25,  0,  0, 34,  0, 23, 19])
Batch Number  1
Data Features tensor([[-16.6045, -17.7573, -17.1969,  ..., -14.4766, -13.6330, -13.4929],
        [-11.6209, -11.7230, -12.8458,  ..., -11.7891, -12.0393, -12.7996],
        [-14.6997,

In [29]:
def initialize_weights(layer):
  if type(layer) == nn.Linear:
    torch.nn.init.xavier_uniform(layer.weight.cuda())
    layer.bias.data.fill_(0.0)

In [30]:
class MLP(nn.Module):

  def __init__(self, network_dimensions):
    super(MLP, self).__init__()
    layers = list()
    self.network_dimensions = network_dimensions
    for i in range(len(network_dimensions)-2):
        if i < 3: 
          layers.append(nn.Linear(self.network_dimensions[i], self.network_dimensions[i+1]).cuda())
          layers.append(nn.BatchNorm1d(self.network_dimensions[i+1]).cuda())
          layers.append(nn.LeakyReLU().cuda())
          layers.append(nn.Dropout(0.25).cuda())
        else:
          layers.append(nn.Linear(self.network_dimensions[i], self.network_dimensions[i+1]).cuda())
          layers.append(nn.LeakyReLU().cuda())
    
    layers.append(nn.Linear(self.network_dimensions[-2], self.network_dimensions[-1]).cuda())
    self.mlp_network = nn.Sequential(*layers)
    self.mlp_network.apply(initialize_weights)

  def forward(self, x):
    return self.mlp_network(x)

In [31]:
def train_model(model, train_data_loader, optimizer, criterion, device):

  model.train()
  total_loss, correct_predictions, total_predictions = 0.0, 0.0, 0.0

  for batch_number, (data_x, data_y) in enumerate(train_data_loader):
    optimizer.zero_grad()
    data_x = data_x.cuda()
    data_y = data_y.cuda()

    data_y_hat = model(data_x)

    _, predicted = torch.max(data_y_hat.data, 1)
    total_predictions += data_y_hat.size(0)
    num_correct = (predicted == data_y).sum().item()
    correct_predictions += num_correct
    
    loss = criterion(data_y_hat, data_y.long())
    total_loss += loss.item()

    loss.backward()
    optimizer.step()

  accuracy = (correct_predictions/total_predictions)
  batch_loss = total_loss/len(train_data_loader)
  return accuracy, batch_loss

In [32]:
def validate_model(model, dev_data_loader, optimizer, criterion, device):
  with torch.no_grad():

    model.eval()
    total_loss, correct_predictions, total_predictions = 0.0, 0.0, 0.0

    for batch_number, (data_x, data_y) in enumerate(dev_data_loader):
      data_x = data_x.cuda()
      data_y = data_y.cuda()

      data_y_hat = model(data_x)

      _, predicted = torch.max(data_y_hat.data, 1)
      total_predictions += data_y_hat.size(0)
      num_correct = (predicted == data_y).sum().item()
      correct_predictions += num_correct

      loss = criterion(data_y_hat, data_y.long()).detach()
      total_loss += loss.item()

    batch_loss = total_loss/len(dev_data_loader)
    accuracy = (correct_predictions/total_predictions)
    return accuracy, batch_loss
    

In [33]:
def predict_labels(model, test_data_loader, device):
  model.eval()

  result = np.array([])
  for batch_number, (data_x, data_y) in enumerate(test_data_loader):
    data_x = data_x.cuda()
    data_y = data_y.cuda()
    print(data_x.shape, data_y.shape)
    data_y_hat = model(data_x)
    _, predicted = torch.max(data_y_hat.data, dim=1)
    result = np.concatenate((result, predicted.cpu().reshape(-1)))
  return result

Setting Model Parameters

In [34]:
# Check if cuda is available
cuda = torch.cuda.is_available()
num_workers = 8 if cuda else 0
print("Cuda = " + str(cuda) + "with num_workers = " + str(num_workers))

context_value = 0
input_size = (2*context_value + 1)*40
output_size = 39
hidden_neurons = [1024, 1024, 2048, 1024, 1024, 512, 256]
network_dimensions = [input_size] + hidden_neurons + [output_size]
model = MLP(network_dimensions)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.0001
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma = 0.1, step_size = 2)
device = torch.device("cuda")
model.cuda()
print(model)

Cuda = Truewith num_workers = 8
MLP(
  (mlp_network): Sequential(
    (0): Linear(in_features=40, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Dropout(p=0.25, inplace=False)
    (4): Linear(in_features=1024, out_features=1024, bias=True)
    (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.01)
    (7): Dropout(p=0.25, inplace=False)
    (8): Linear(in_features=1024, out_features=2048, bias=True)
    (9): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.01)
    (11): Dropout(p=0.25, inplace=False)
    (12): Linear(in_features=2048, out_features=1024, bias=True)
    (13): LeakyReLU(negative_slope=0.01)
    (14): Linear(in_features=1024, out_features=1024, bias=True)
    (15): LeakyReLU(negative_slope=0.01)
    (16): Linear(

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
train_loss = train_model(model, train_loader, optimizer, criterion, device)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [36]:
print(train_loss)

(0.7682322705601907, 0.9954450635929926)


In [37]:
dev_accuracy, dev_loss = validate_model(model, dev_loader, optimizer, criterion, device)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [38]:
MODEL_VERSION = 3
def save_model(model, optimizer, scheduler, epoch, train_loss, dev_loss, dev_accuracy):
  model_epoch_path = f"{drivepath_final}/models/model_{MODEL_VERSION}_{epoch}"
  torch.save({
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'scheduler_state_dict': scheduler.state_dict(),
      'train_loss': train_loss,
      'validation_loss': dev_loss,
      'validation_accuracy': dev_accuracy
  },model_epoch_path)

In [39]:
training_loss, validation_loss, validation_accuracy = list(), list(), list()
num_epochs = 100

for epoch in range(0, num_epochs):
  train_accuracy, train_loss = train_model(model, train_loader, optimizer, criterion, device)
  dev_accuracy, dev_loss = validate_model(model, dev_loader, optimizer, criterion, device)
  scheduler.step(dev_loss)
  training_loss.append(train_loss)
  validation_loss.append(dev_loss)
  validation_accuracy.append(dev_accuracy)
  print("Completed Epoch " + str(epoch+1))
  print("Training Loss ", train_loss)
  print("Training Accuracy ", train_accuracy)
  print("Validation Loss ", dev_loss)
  print("Validation Accuracy ", dev_accuracy)
  print("-----------------------------------------------------------")
  save_model(model, optimizer, scheduler, epoch, train_loss, dev_loss, dev_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarni

Completed Epoch 1
Training Loss  0.6784550605321812
Training Accuracy  0.84375
Validation Loss  0.8244621583157116
Validation Accuracy  0.8235243055555556
-----------------------------------------------------------
Completed Epoch 2
Training Loss  0.6114533930768273
Training Accuracy  0.859877830750894
Validation Loss  0.8014108224047555
Validation Accuracy  0.8348090277777778
-----------------------------------------------------------
Completed Epoch 3
Training Loss  0.5806107842879585
Training Accuracy  0.8675506555423123
Validation Loss  0.766973244978322
Validation Accuracy  0.8369791666666667
-----------------------------------------------------------
Completed Epoch 4
Training Loss  0.5510427290762423
Training Accuracy  0.8743668057210966
Validation Loss  0.7710385837488705
Validation Accuracy  0.8336805555555555
-----------------------------------------------------------
Completed Epoch 5
Training Loss  0.5309190908458008
Training Accuracy  0.8785011918951132
Validation Loss  0.